In [2]:
# !pip install cx_Oracle  #데이터베이스 오라클 연동 패키지 다운

In [3]:
import cx_Oracle

In [4]:
connection = cx_Oracle.connect("hr", "hoseo","localhost:1521/xe") # 사용자이름, 비번, 서버위치

In [5]:
cursor = connection.cursor()  # cursor : 명령 처리기 

In [6]:
cursor.execute("select * from departments") #SQL실행 + 결과를 내부에 저장

<cx_Oracle.Cursor on <cx_Oracle.Connection to hr@localhost:1521/xe>>

In [7]:
for did, dname, mid, lid in cursor:
    print("[{0}][{1}][{2}][{3}]".format(did, dname, mid, lid))

[10][Administration][200][1700]
[20][Marketing][201][1800]
[30][Purchasing][114][1700]
[40][Human Resources][203][2400]
[50][Shipping][121][1500]
[60][IT][103][1400]
[70][Public Relations][204][2700]
[80][Sales][145][2500]
[90][Executive][100][1700]
[100][Finance][108][1700]
[110][Accounting][205][1700]
[120][Treasury][None][1700]
[130][Corporate Tax][None][1700]
[140][Control And Credit][None][1700]
[150][Shareholder Services][None][1700]
[160][Benefits][None][1700]
[170][Manufacturing][None][1700]
[180][Construction][None][1700]
[190][Contracting][None][1700]
[200][Operations][None][1700]
[210][IT Support][None][1700]
[220][NOC][None][1700]
[230][IT Helpdesk][None][1700]
[240][Government Sales][None][1700]
[250][Retail Sales][None][1700]
[260][Recruiting][None][1700]
[270][Payroll][None][1700]


In [8]:
# 연결종료
cursor.close()
connection.close()

In [10]:
import pandas as pd     #데이터프레임

In [9]:
connection = cx_Oracle.connect("hr", "hoseo" , "localhost:1521/xe") # lacalhost == 127.0.0.1

In [11]:
departments = pd.read_sql("select * from departments", connection)

In [12]:
departments.head()

,DEPARTMENT_ID,DEPARTMENT_NAME,MANAGER_ID,LOCATION_ID
0,10,Administration,200.0,1700
1,20,Marketing,201.0,1800
2,30,Purchasing,114.0,1700
3,40,Human Resources,203.0,2400
4,50,Shipping,121.0,1500


In [13]:
connection.close()

In [17]:
connection = cx_Oracle.connect("hr", "hoseo", "localhost:1521/xe")
cursor = connection.cursor()

cursor.execute("""select employee_id, first_name, last_name, hire_date 
                 from employees
                 where lower(first_name) like '%x%'""")   # 효율적이지 않은 방식
for eid, fn, ln, hd in cursor:
    print("[{0}][{1}][{2}][{3}]".format(eid, fn, ln, hd))

cursor.close()
connection.close()

[102][Lex][De Haan][2001-01-13 00:00:00]
[103][Alexander][Hunold][2006-01-03 00:00:00]
[115][Alexander][Khoo][2003-05-18 00:00:00]
[185][Alexis][Bull][2005-02-20 00:00:00]


In [19]:
connection = cx_Oracle.connect("hr", "hoseo", "localhost:1521/xe")
cursor = connection.cursor()

cursor.execute("""select employee_id, first_name, last_name, hire_date 
                 from employees
                 where lower(first_name) like :fn """, fn='%x%') 
#SQL에서 ":이름"은 변수   / :fn <- 변수 / 데이터를 바꿔도 처리가 가능
                
for eid, fn, ln, hd in cursor:
    print("[{0}][{1}][{2}][{3}]".format(eid, fn, ln, hd))

cursor.close()
connection.close()

[102][Lex][De Haan][2001-01-13 00:00:00]
[103][Alexander][Hunold][2006-01-03 00:00:00]
[115][Alexander][Khoo][2003-05-18 00:00:00]
[185][Alexis][Bull][2005-02-20 00:00:00]


In [28]:
name_key = input("name key : ")   # 명령행에서 사용자 입력

connection = cx_Oracle.connect("hr", "hoseo", "localhost:1521/xe")

employees=pd.read_sql("""select employee_id, first_name, last_name, hire_date 
                         from employees
                         where lower(first_name) like :fn """,
                         connection, 
                         params={'fn':'%{0}%'.format(name_key) }) 

connection.close()

employees

name key :  ch


,EMPLOYEE_ID,FIRST_NAME,LAST_NAME,HIRE_DATE
0,134,Michael,Rogers,2006-08-26
1,153,Christopher,Olsen,2006-03-30
2,179,Charles,Johnson,2008-01-04
3,201,Michael,Hartstein,2004-02-17


In [31]:
country_key = input("country : ")   # 명령행에서 사용자 입력

connection = cx_Oracle.connect("hr", "hoseo", "localhost:1521/xe")

employees=pd.read_sql("""select e.first_name, e.last_name, e.email, c.country_name
                         from employees e, departments d, locations l, countries c
                         where e.department_id = d.department_id and
                               d.location_id = l.location_id and
                               l.country_id = c.country_id and
                               lower(c.country_name) like : country""",
                         connection, 
                         params={'country':'%{0}%'.format(country_key) }) 

connection.close()

employees

country :  canada


,FIRST_NAME,LAST_NAME,EMAIL,COUNTRY_NAME
0,Michael,Hartstein,MHARTSTE,Canada
1,Pat,Fay,PFAY,Canada


In [1]:
import cx_Oracle
import numpy as np
import pandas as pd

In [5]:
# sql 데이터에 데이터 추가하기
conn = cx_Oracle.connect("hr", "hoseo", "127.0.01:1521/xe") 
cursor=conn.cursor()

sql="""insert into jobs(job_id, job_title,min_salary, max_salary)
        values
        (:id, :title, :mins, :maxs)"""

cursor.execute(sql, id='db eng', title='database engineer', mins=4000, maxs=10000)

conn.commit()   

cursor.close()
conn.close()                     

In [14]:
# sql 데이터에 데이터 조회
conn = cx_Oracle.connect("hr", "hoseo", "127.0.01:1521/xe") 
cursor=conn.cursor()

sql="""select * from jobs
        where job_id = 'db eng'"""

jobs = pd.read_sql(sql, conn)


cursor.close()
conn.close()

jobs

,JOB_ID,JOB_TITLE,MIN_SALARY,MAX_SALARY


In [11]:
# sql 데이터 업데이트
conn = cx_Oracle.connect("hr", "hoseo", "127.0.01:1521/xe") 
cursor=conn.cursor()

sql="""update jobs
        set min_salary= :mins, max_salary = :maxs
        where job_id = :id """

cursor.execute(sql, mins=5000, maxs=12000 , id='db eng')

conn.commit()   

cursor.close()
conn.close()

In [13]:
# sql 데이터 삭제하기

conn = cx_Oracle.connect("hr", "hoseo", "127.0.01:1521/xe") 
cursor=conn.cursor()

sql="""delete from jobs
        where job_id = :id """

cursor.execute(sql, id='db eng')

conn.commit()   #최종 저장

cursor.close()
conn.close()